# 1.5 Command Line Interface (CLI)

The following section demonstrates how to use the **signac** command line interface (CLI).
The CLI allows you to interact with your data space without python, which may be advantageous in various situations, e.g., for scripting or data exploration.

You will find that for many of the functions introduced earlier there is an equivalent CLI command.

The CLI is accessed via the top-level ``signac`` command.
You can get help about the various functions with the `-h` or `--help` argument.

In [ ]:
%%bash
signac --help

To interact with a project on the command line, the current working directory needs to be within or below the project's root directory.
Let's start by reseting the designated project root directory for this section of the tutorial.

In [ ]:
%rm -rf projects/tutorial/cli
%mkdir -p projects/tutorial/cli

Next we switch the current working directory to the project root directory.

In [ ]:
%cd projects/tutorial/cli

Then we initialize the project.

In [ ]:
%%bash
signac init TutorialCLIProject

We can verify the project configuration using the `signac project` command.

In [ ]:
%%bash
signac project
signac project --workspace

We access a job by providing the state point on the command line in JSON format †).

†) *The JSON format requires double quotes for keys.*

In [ ]:
%%bash
signac job '{"kT": 1.0, "p": 1.0, "N": 1000}'

By default this will print the associated *job id* to STDOUT.

Instead of the *job id*, we can also get the path to the job's workspace.

In [ ]:
%%bash
signac job '{"kT": 1.0, "p": 1.0, "N": 1000}' --workspace

Please not, that obtaining the path in this way does not necessarily mean that the path exists.
However, we can initialize the job and create the workspace using the `-c` or `--create` argument.

In [ ]:
%%bash
signac job '{"kT": 1.0, "p": 1.0, "N": 1000}' --create

We can use the `signac statepoint` command to get the statepoint associated with the *initialized* job.

In [ ]:
%%bash
signac statepoint ee617ad585a90809947709a7a45dda9a

Usually we will not provide statepoints on the command line, but read them from a file.
Let's create a statepoint file with one statepoint:

In [ ]:
%%bash
echo '{"kT": 1.0, "p": 0.1, "N": 1000}' > statepoint.txt
cat statepoint.txt

We can pipe the content of this file into the **signac** CLI to get the corresponding *job id*.

In [ ]:
%%bash
cat statepoint.txt | signac job

We will reproduce the ideal gas project from section 1.1 to generate some data for the following examples.

In [ ]:
import signac

def V_idg(N, p, kT):
    return N * kT / p

project = signac.get_project()
for p in 0.1, 1.0, 10.0:
    sp = {'p': p, 'kT': 1.0, 'N': 1000}
    job = project.open_job(sp)
    job.document['V'] = V_idg(**sp)

We can use the `signac find` command to find all jobs within our project's workspace.

In [ ]:
%%bash
signac find

Just like with `project.find_jobs()` we can provide a filter argument to find a subset of jobs matching the given filter.
Here we get all jobs with a pressure of 0.1:

In [ ]:
%%bash
signac find '{"p": 0.1}'

In this example, that is of course only one job.

Similarly, we can also filter based on information in the job document. Here, we find all jobs that have a volume corresponding to a pressure of 1 (volume = 1000*1/1 = 1000).

In [ ]:
%%bash
signac find --doc-filter '{"V": 1000.0}'

Once again, this only returns one job in this case.

We can pipe `signac find` results into `signac statepoint` with `xargs` to resolve the statepoints.

In [ ]:
%%bash
signac find | xargs signac statepoint

You will have noticed that each time we execute a *find* operation the data space is indexed anew.

This is no problem for small data spaces, however especially for larger data spaces, where the indexing process may be substantially expensive it's advantageous to cache the index in a file.

In [ ]:
%%bash
signac project --index > index.txt

This index file can be used in conjunction with all functions that require a data space index, for example `signac find`:

In [ ]:
%%bash
signac find -i index.txt '{"p": 0.1}'

Or for instance when creating a linked view.

In [ ]:
%%bash
signac view -i index.txt ./view

The `signac view` command works exactly like `project.create_linked_view` such that the `./view` directory now contains a linked view to the job workspaces.

In [ ]:
%ls view

This concludes the first chapter of the tutorial.
The next chapter introduces a few more advanced topics.

[Return to index](index.ipynb)